##### Import

In [1]:
import pandas as pd 
import os
from pathlib import Path
import numpy as np

##### Crée un dictionnaire avec tous les datasets 

In [2]:
csvDict = {}

for subdir, dirs, files in os.walk(r'Datasets_Kaggle/'):
    for filename in files:
        filepath = subdir + os.sep + filename
        if filepath.endswith(".csv"):
            #if "MN5_AROME" in filename:
            csvDict[filename[:-4]] = pd.read_csv(Path(filepath),sep=",")

for i in csvDict.keys():
    print (i)

circuits
constructors
constructor_results
drivers
lap_times
pit_stops
qualifying
races
results
status


##### Affiche le nom de toutes les colonnes de tous les datasets

In [3]:
for dataset in csvDict:
    print("---", dataset)
    for col in csvDict[dataset]:
        print(col)

--- circuits
circuitId
circuitRef
name
location
country
lat
lng
alt
url
--- constructors
constructorId
constructorRef
name
nationality
url
--- constructor_results
constructorResultsId
raceId
constructorId
points
status
--- drivers
driverId
driverRef
number
code
forename
surname
dob
nationality
url
--- lap_times
raceId
driverId
lap
position
time
milliseconds
--- pit_stops
raceId
driverId
stop
lap
time
duration
milliseconds
--- qualifying
qualifyId
raceId
driverId
constructorId
number
position
q1
q2
q3
--- races
raceId
year
round
circuitId
name
date
time
url
--- results
resultId
raceId
driverId
constructorId
number
grid
position
positionText
positionOrder
points
laps
time
milliseconds
fastestLap
rank
fastestLapTime
fastestLapSpeed
statusId
--- status
statusId
status


##### Pour visualiser un dataset parmi le dictionnaire de datasets

In [4]:
csvDict["races"].head()

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


##### Vérifie que les dates soient bien au format date compris par la librairie pandas

In [5]:
csvDict["races"]["date"] = pd.to_datetime(csvDict["races"]["date"])

##### Choses à faire dans le nettoyage (à première vue)

- Se poser la question d'utiliser la totalité des données historique que l'on a à disposition
- Supprimer les URL si nécessaire 
---
- Voir au niveau des écuries constructeurs si ce ne sont pas les mêmes car parfois il y a juste un changement de nom d'écurie mais c'est toujours la même (dataset constructors)
- Lier les résultats des constructeurs au dataset constructors (voir comment faire, éventuellement un dataset pour chaque écurie avec ses résultats par courses)
- Voir si il y a une différence entre position et positionText (dataset constructor_standings)
- Voir la différence de lignes à quoi elle correspond entre le dataset constructor_results et constructor_standings
- Lier les constructeurs aux résultats obtenus par leur pilote avec leur statusID (présent dans le dataset status)
---
- Faire le lien entre drivers et driver_standings
- Lier les pilotes à leur résultat en qualification
- Lier les pilotes à leur résultat en course avec leur statusID (présent dans le dataset status)
---
- Voir quel intérêt dans un modèle de Machine Learning il peut y avoir à étudier le temps de chaque pilote à chaque tour par course (sachant que dans le dataset results, on a déjà les meilleurs temps de chaque pilote de la course)
- Voir quel intérêt en ML de conserver les pit_stops
---
- Lier les datasets circuits et races
---
- seasons ? utilité de conserver ce dataset ? 

In [6]:
csvDict["circuits"].head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,\N,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,\N,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,\N,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,\N,http://en.wikipedia.org/wiki/Istanbul_Park


In [7]:
csvDict["circuits"].alt.unique()

array(['10', '\\N'], dtype=object)

In [8]:
circuits = csvDict["circuits"]
circuits.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,\N,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,\N,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,\N,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,\N,http://en.wikipedia.org/wiki/Istanbul_Park


In [9]:
"""for i in range (0, len(circuits)):
    for circuits[["alt"]] in circuits:
        if "\\N" in circuits[["alt"]]:
            print(i)"""

'for i in range (0, len(circuits)):\n    for circuits[["alt"]] in circuits:\n        if "\\N" in circuits[["alt"]]:\n            print(i)'

In [10]:
circuits["alt"] = circuits["alt"].replace('\\N', 0)

In [11]:
circuits = circuits.drop(columns = ["url"], axis=1)
circuits

,circuitId,circuitRef,name,location,country,lat,lng,alt
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,0
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,0
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,0
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,0
...,...,...,...,...,...,...,...,...
71,72,port_imperial,Port Imperial Street Circuit,New Jersey,USA,40.77690,-74.01110,0
72,73,BAK,Baku City Circuit,Baku,Azerbaijan,40.37250,49.85330,0
73,74,hanoi,Hanoi Street Circuit,Hanoi,Vietnam,21.01660,105.76600,0
74,75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,0


In [12]:
for i in range (0, len(circuits)):
    for col in circuits:
        if "\\N" in col:
            print(i)

In [13]:
#circuits.to_csv("circuits_Clean.csv", sep=';')

In [14]:
cons = csvDict["constructors"]
cons = cons.drop(columns = ["url"], axis=1)
cons

,constructorId,constructorRef,name,nationality
0,1,mclaren,McLaren,British
1,2,bmw_sauber,BMW Sauber,German
2,3,williams,Williams,British
3,4,renault,Renault,French
4,5,toro_rosso,Toro Rosso,Italian
...,...,...,...,...
206,209,manor,Manor Marussia,British
207,210,haas,Haas F1 Team,American
208,211,racing_point,Racing Point,British
209,212,alpha_tauri,Scuderia Alpha Tauri,Italian


In [15]:
#cons.to_csv("constructeurs_Clean.csv", sep=';')

In [16]:
cons_res = csvDict["constructor_results"]
cons_res = cons_res.drop(columns = ["status"], axis=1)

In [17]:
#cons_res.to_csv("contructeurs_results_Clean.csv", sep=';')

In [18]:
dri = csvDict["drivers"]
dri

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
...,...,...,...,...,...,...,...,...,...
843,845,sirotkin,35,SIR,Sergey,Sirotkin,1995-08-27,Russian,http://en.wikipedia.org/wiki/Sergey_Sirotkin_(...
844,846,norris,4,NOR,Lando,Norris,1999-11-13,British,http://en.wikipedia.org/wiki/Lando_Norris
845,847,russell,63,RUS,George,Russell,1998-02-15,British,http://en.wikipedia.org/wiki/George_Russell_%2...
846,848,albon,23,ALB,Alexander,Albon,1996-03-23,Thai,http://en.wikipedia.org/wiki/Alexander_Albon


In [19]:
dri = dri.drop(columns = ["url"], axis=1)

s = dri.loc[dri["code"] == '\\N']
s

,driverId,driverRef,number,code,forename,surname,dob,nationality
42,43,matta,\N,\N,Cristiano,da Matta,1973-09-19,Brazilian
43,44,panis,\N,\N,Olivier,Panis,1966-09-02,French
44,45,pantano,\N,\N,Giorgio,Pantano,1979-02-04,Italian
45,46,bruni,\N,\N,Gianmaria,Bruni,1981-05-30,Italian
46,47,baumgartner,\N,\N,Zsolt,Baumgartner,1981-01-01,Hungarian
...,...,...,...,...,...,...,...,...
802,802,serafini,\N,\N,Dorino,Serafini,1909-07-22,Italian
803,803,cantrell,\N,\N,Bill,Cantrell,1908-01-31,American
804,804,mantz,\N,\N,Johnny,Mantz,1918-09-18,American
805,805,kladis,\N,\N,Danny,Kladis,1917-02-10,American


In [20]:
for i in range (len(dri)):
    first_three_letters = dri["driverRef"][i][:3]
    dri["code"][i] = first_three_letters

dri
    

C:\Users\maxou\Anaconda3\envs\Formula1\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,driverId,driverRef,number,code,forename,surname,dob,nationality
0,1,hamilton,44,ham,Lewis,Hamilton,1985-01-07,British
1,2,heidfeld,\N,hei,Nick,Heidfeld,1977-05-10,German
2,3,rosberg,6,ros,Nico,Rosberg,1985-06-27,German
3,4,alonso,14,alo,Fernando,Alonso,1981-07-29,Spanish
4,5,kovalainen,\N,kov,Heikki,Kovalainen,1981-10-19,Finnish
...,...,...,...,...,...,...,...,...
843,845,sirotkin,35,sir,Sergey,Sirotkin,1995-08-27,Russian
844,846,norris,4,nor,Lando,Norris,1999-11-13,British
845,847,russell,63,rus,George,Russell,1998-02-15,British
846,848,albon,23,alb,Alexander,Albon,1996-03-23,Thai


In [21]:
dri["number"] = dri["number"].replace('\\N', np.nan )
dri

,driverId,driverRef,number,code,forename,surname,dob,nationality
0,1,hamilton,44,ham,Lewis,Hamilton,1985-01-07,British
1,2,heidfeld,NaN,hei,Nick,Heidfeld,1977-05-10,German
2,3,rosberg,6,ros,Nico,Rosberg,1985-06-27,German
3,4,alonso,14,alo,Fernando,Alonso,1981-07-29,Spanish
4,5,kovalainen,NaN,kov,Heikki,Kovalainen,1981-10-19,Finnish
...,...,...,...,...,...,...,...,...
843,845,sirotkin,35,sir,Sergey,Sirotkin,1995-08-27,Russian
844,846,norris,4,nor,Lando,Norris,1999-11-13,British
845,847,russell,63,rus,George,Russell,1998-02-15,British
846,848,albon,23,alb,Alexander,Albon,1996-03-23,Thai


In [22]:
#dri.to_csv("drivers_Clean.csv", sep=';')

In [23]:
lap = csvDict["lap_times"]
lap

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342
...,...,...,...,...,...,...
479386,1036,849,60,19,1:22.030,82030
479387,1036,849,61,19,1:22.422,82422
479388,1036,849,62,18,1:22.909,82909
479389,1036,849,63,18,1:23.479,83479


In [24]:
lap = lap.drop(columns = ["milliseconds"], axis=1)

In [25]:
a = lap.loc[lap["time"] == '\\N']
a

,raceId,driverId,lap,position,time


In [26]:
lap = lap.dropna(subset=["lap","position","time"])
lap

,raceId,driverId,lap,position,time
0,841,20,1,1,1:38.109
1,841,20,2,1,1:33.006
2,841,20,3,1,1:32.713
3,841,20,4,1,1:32.803
4,841,20,5,1,1:32.342
...,...,...,...,...,...
479386,1036,849,60,19,1:22.030
479387,1036,849,61,19,1:22.422
479388,1036,849,62,18,1:22.909
479389,1036,849,63,18,1:23.479


In [27]:
#lap.to_csv("lap_times_Clean.csv", sep=';')